#### OBJETIVO: Contruccion de modelos Predictivos de PM2.5 con CV aleatorio


In [2]:
#Librerias
import pandas as pd
import numpy as np

from datetime import datetime

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
import joblib
import pandas as pd
import numpy as np
import time
import xgboost as xgb


print("Librerias ok")

Librerias ok


### Variables descartadas:
- SP: DEM, t2m (14v)
- ST: sp (15v)
- BA: t2m (15v)
- MD: - (16v)
- MX: - (16v)

In [4]:
#Funcion para evaluar el modelo

def evaluar_modelo(pred, y_real):
    df = pd.DataFrame({
        "pred": pred,
        "real": y_real
    })

    df = df[df["pred"] > 0]

    r2 = np.corrcoef(df["pred"], df["real"])[0, 1] ** 2
    pearson = np.corrcoef(df["pred"], df["real"])[0, 1]
    rmse = np.sqrt(np.mean((df["pred"] - df["real"]) ** 2))
    bias = np.mean(df["pred"] - df["real"])

    return pd.DataFrame({
        "R2": [round(r2, 5)],
        "Pearson": [round(pearson, 3)],
        "RMSE": [round(rmse, 3)],
        "Bias": [round(bias, 3)],
        "Min_Pred": [round(df["pred"].min(), 3)],
        "Max_Pred": [round(df["pred"].max(), 3)]
    })


In [15]:
### -------------- SVR --------------------------
#Ingreso de datos
estacion = "MX"
modelo = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_train_{estacion}.csv"
)
test_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_test_{estacion}.csv"
)
# print(train_data.columns)
# Variables
features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean",  "d2m_mean",  "sp_mean", "t2m_mean",
    "v10_mean", "u10_mean", "tp_mean",  "dayWeek","DEM"
]
#### Seleccionar variables del dataset
X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

# Pipeline, paso a paso del modelo
pipeline_svr = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(
        kernel="rbf",
        C=10,
        epsilon=0.1,
        gamma="scale"
    ))
])

#Cross validation solo evaluacion
pred_cv = cross_val_predict(
    pipeline_svr,
    X_train,
    y_train,
    cv=10,
    n_jobs=-1
)
#Evaluar el desempñeo del train
resultados_cv = evaluar_modelo(pred_cv, y_train)

print("Resultados CV (train):")
print(resultados_cv)

#Se ajustan los datos
pipeline_svr.fit(X_train, y_train)

#Predicho del test 
pred_test = pipeline_svr.predict(X_test)
#Evaluar el desempñeo del test
resultados_test = evaluar_modelo(pred_test, y_test)
#Resultados del test
print("Resultados TEST:")
print(resultados_test)

#Guardar el modelo
ruta_modelo = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/01-SVR-CV10-M{modelo}-{estacion}_220126.joblib"

joblib.dump(pipeline_svr, ruta_modelo)




Resultados CV (train):
        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.58534    0.765  6.605 -0.557     3.677    77.068
Resultados TEST:
        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.64847    0.805  5.976 -0.682     2.167    73.259


['D:/Josefina/Proyectos/Tesis/MX/modelos_py/01-SVR-CV10-M1-MX_220126.joblib']

In [ ]:
#Probamos guardar el modelo y levantarlo desde el python para evaluar el desempeñ
#Funciona ok
estacion = "SP"
modelo = "1"
ruta_modelo = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/01-SVR-CV10-M{modelo}-{estacion}_220126.joblib"


modelo_cargado = joblib.load(ruta_modelo)

pred_test_loaded = modelo_cargado.predict(X_test)

evaluar_modelo(pred_test_loaded, y_test)


,R2,Pearson,RMSE,Bias,Min_Pred,Max_Pred
0,0.77673,0.881,7.922,-0.986,4.1,81.426


In [20]:
### -------------- Extra trees --------------------------
#Ingreso de datos
estacion = "MX"
modelo = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_train_{estacion}.csv"
)
test_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_test_{estacion}.csv"
)

# Variables
features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean",  "d2m_mean", "sp_mean", "t2m_mean",
    "v10_mean", "u10_mean", "tp_mean",  "dayWeek","DEM"
]
#### Seleccionar variables del dataset
X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Generar modelo
et_model = ExtraTreesRegressor(
    n_estimators=500,        # num.trees
    max_features=5,          # mtry
    min_samples_leaf=5,      # min.node.size
    random_state=123,
    n_jobs=-1
)

#Validacion crtuzada del modelo
pred_cv = cross_val_predict(
    et_model,
    X_train,
    y_train,
    cv=10,
    n_jobs=-1
)
#Resultados del train
resultados_ET_cv = evaluar_modelo(pred_cv, y_train)

print("Resultados CV (train):")
print(resultados_ET_cv)
#Entrenar el modelo final
et_model.fit(X_train, y_train)

#Evaluacion del modelo final
pred_test = et_model.predict(X_test)

resultados_test = evaluar_modelo(pred_test, y_test)

print("Resultados TEST:")
print(resultados_test)

#Guardar el modelo
ruta_modelo = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/01-ET-CV10-M{modelo}-{estacion}_220126.joblib"

joblib.dump(et_model, ruta_modelo)


Resultados CV (train):
        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.62706    0.792  6.487  0.074     9.356     72.55
Resultados TEST:
        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.67816    0.824  5.948  0.059     7.141    74.237


['D:/Josefina/Proyectos/Tesis/MX/modelos_py/01-ET-CV10-M1-MX_220126.joblib']

In [ ]:
# Abrimos el modelo que fue guardado antes, para probar
ruta_modelo = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos/01-ET-CV10-M{modelo}-{estacion}.joblib"
joblib.dump(et_model, ruta_modelo)

modelo_cargado = joblib.load(ruta_modelo)
#Esta evaluacion es lo mismo que lo anterior
pred_test_loaded = modelo_cargado.predict(X_test)

evaluar_modelo(pred_test_loaded, y_test)
#Algunos de los datos de los modelos
print("Número de árboles:")
print(et_model.n_estimators)

print("\nMin node size:")
print(et_model.min_samples_leaf)

print("\nMtry (max_features):")
print(et_model.max_features)

print("\nImportancia de variables:")
importancias = pd.Series(
    et_model.feature_importances_,
    index=features
).sort_values(ascending=False)

print(importancias)


Número de árboles:
500

Min node size:
5

Mtry (max_features):
5

Importancia de variables:
blh_mean        0.311582
u10_mean        0.177605
SO2SMASS_dia    0.093457
d2m_mean        0.083799
BCSMASS_dia     0.073595
sp_mean         0.047326
DEM             0.040959
ndvi            0.039590
AOD_055         0.028953
v10_mean        0.028670
dayWeek         0.018283
SSSMASS_dia     0.017563
DUSMASS_dia     0.015409
SO4SMASS_dia    0.012241
tp_mean         0.010970
dtype: float64


In [26]:
### -------------- Random Forest --------------------------
#Ingreso de datos
estacion = "MX"
modelo = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_train_{estacion}.csv"
)
test_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_test_{estacion}.csv"
)

# Variables
features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean",  "d2m_mean", "sp_mean", "t2m_mean",
    "v10_mean", "u10_mean", "tp_mean",  "dayWeek","DEM"
]
#### Seleccionar variables del dataset
X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Generar modelo
rf_model = RandomForestRegressor(
    n_estimators=500,        # ntree
    max_features=5,          # mtry (fijado explícitamente)
    min_samples_leaf=5,      # nodesize
    random_state=123,
    n_jobs=-1
)
start_time = time.time()
# prediccion con cross validation
pred_cv = cross_val_predict(
    rf_model,
    X_train,
    y_train,
    cv=10,
    n_jobs=-1
)
#Vemos el tiempo que tarda en generar el modelo
end_time = time.time()
print(f"Tiempo de entrenamiento (CV): {round(end_time - start_time, 2)} segundos")
#Evaluar el desempeño del train
resultados_RF_cv = evaluar_modelo(pred_cv, y_train)

print("Resultados CV (train):")
print(resultados_RF_cv)
rf_model.fit(X_train, y_train)
pred_test = rf_model.predict(X_test)
#Evaluar el desempeño del test
resultados_test = evaluar_modelo(pred_test, y_test)

print("Resultados TEST:")
print(resultados_test)


#Guardar el modelo
ruta_modelo = (
    f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/"
    f"01-RF-CV10-M{modelo}-{estacion}_220126.joblib"
)

joblib.dump(rf_model, ruta_modelo)


Tiempo de entrenamiento (CV): 170.32 segundos
Resultados CV (train):
        R2  Pearson  RMSE   Bias  Min_Pred  Max_Pred
0  0.64164    0.801  6.22  0.163     8.513    78.495
Resultados TEST:
        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.69182    0.832  5.685  0.056     6.359    79.287


['D:/Josefina/Proyectos/Tesis/MX/modelos_py/01-RF-CV10-M1-MX_220126.joblib']

In [ ]:
# Probamos guardar y subir el modelo
modelo_cargado = joblib.load(ruta_modelo)

pred_test_loaded = modelo_cargado.predict(X_test)

evaluar_modelo(pred_test_loaded, y_test)

print("Número de árboles (ntree):", rf_model.n_estimators)
print("mtry (max_features):", rf_model.max_features)
print("nodesize (min_samples_leaf):", rf_model.min_samples_leaf)

importancias = pd.Series(
    rf_model.feature_importances_,
    index=features
).sort_values(ascending=False)

print("\nImportancia de variables:")
print(importancias)


Número de árboles (ntree): 500
mtry (max_features): 5
nodesize (min_samples_leaf): 5

Importancia de variables:
blh_mean        0.304127
u10_mean        0.147912
SO2SMASS_dia    0.103232
BCSMASS_dia     0.098870
d2m_mean        0.069367
sp_mean         0.052383
ndvi            0.043623
AOD_055         0.034610
DEM             0.030054
tp_mean         0.023994
v10_mean        0.023284
DUSMASS_dia     0.019495
SO4SMASS_dia    0.018697
SSSMASS_dia     0.016279
dayWeek         0.014072
dtype: float64


In [31]:
### -------------- XGB --------------------------
#Ingreso de datos
estacion = "MX"
modelo = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_train_{estacion}.csv"
)
test_data = pd.read_csv(
    f"{base_dir}Modelo_{modelo}/M{modelo}_test_{estacion}.csv"
)

# Variables
features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean",  "d2m_mean",  "sp_mean", "t2m_mean",
    "v10_mean", "u10_mean", "tp_mean",  "dayWeek","DEM"
]
#### Seleccionar variables del dataset
X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Generar modelo
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
#Setear los parametros
params = {
    "booster": "gbtree",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.3,
    "max_depth": 6,
    "gamma": 0,
    "subsample": 0.8,
    "colsample_bytree": 1,
    "min_child_weight": 1,
    "seed": 123
}

#Vemos cuanto tarda el modelo en entrenamiento
start_time = time.time()
# Entrenar el modelo
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,
    nfold=10,
    early_stopping_rounds=20,
    verbose_eval=True
)
#Termino el modelo
end_time = time.time()
#print(f"Tiempo CV XGB: {round(end_time - start_time, 2)} segundos")

#Numero de rondas
best_nrounds = len(cv_results)
print("Número óptimo de rondas:", best_nrounds)

xgb_cv_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=best_nrounds
)


pred_test = xgb_cv_model.predict(dtest)

resultados_XGB = evaluar_modelo(pred_test, y_test)
print("Resultados TEST XGB:")
print(resultados_XGB)

ruta_modelo = (
    f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/"
    f"01-XGB-CV-M{modelo}-{estacion}_220126.joblib"
)

joblib.dump(xgb_cv_model, ruta_modelo)



[0]	train-rmse:8.87393+0.02979	test-rmse:8.97600+0.31394
[1]	train-rmse:7.97636+0.04289	test-rmse:8.17959+0.27748
[2]	train-rmse:7.34057+0.02671	test-rmse:7.64394+0.25437
[3]	train-rmse:6.88908+0.02400	test-rmse:7.29048+0.25826
[4]	train-rmse:6.54125+0.03283	test-rmse:7.01940+0.25088
[5]	train-rmse:6.28499+0.04037	test-rmse:6.83785+0.25483
[6]	train-rmse:6.06047+0.03896	test-rmse:6.67768+0.26541
[7]	train-rmse:5.85956+0.05291	test-rmse:6.53043+0.25589
[8]	train-rmse:5.70125+0.05156	test-rmse:6.42497+0.25624
[9]	train-rmse:5.56497+0.04737	test-rmse:6.33953+0.24141
[10]	train-rmse:5.44082+0.04222	test-rmse:6.26504+0.23414
[11]	train-rmse:5.32050+0.04821	test-rmse:6.18872+0.23737
[12]	train-rmse:5.22705+0.05500	test-rmse:6.13550+0.24183
[13]	train-rmse:5.13576+0.05424	test-rmse:6.08848+0.25392
[14]	train-rmse:5.06089+0.05667	test-rmse:6.05543+0.24664
[15]	train-rmse:4.99466+0.05777	test-rmse:6.01929+0.23701
[16]	train-rmse:4.93031+0.04980	test-rmse:5.98880+0.23148
[17]	train-rmse:4.87852+

['D:/Josefina/Proyectos/Tesis/MX/modelos_py/01-XGB-CV-M1-MX_220126.joblib']

In [ ]:
# Vemos coeficientes del modelo
modelo_cargado = joblib.load(ruta_modelo)

pred_loaded = modelo_cargado.predict(dtest)
print(evaluar_modelo(pred_loaded, y_test))

print("Parámetros XGB:")
print(params)

print("\nNúmero de rondas usadas:")
print(best_nrounds)


        R2  Pearson   RMSE   Bias  Min_Pred    Max_Pred
0  0.83914    0.916  6.613  0.098     2.894  135.270004
Parámetros XGB:
{'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.3, 'max_depth': 6, 'gamma': 0, 'subsample': 0.8, 'colsample_bytree': 1, 'min_child_weight': 1, 'seed': 123}

Número de rondas usadas:
568
